<a href="https://colab.research.google.com/github/kaeyang/KDE-Analysts/blob/main/KDE_Analysts_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Imports and Setup

In [ ]:
#install geopandas and descartes
!pip install geopandas
!pip install descartes

# Import 3rd party libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import datetime
import numpy as np
import geopandas as gpd

# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

!pip install geopandas
!pip install descartes

pd.options.mode.chained_assignment = None  # default='warn'

from google.colab import drive
drive.mount('/content/drive/')


     |████████████████████████████████| 1.0MB 11.8MB/s 
     |████████████████████████████████| 15.3MB 235kB/s 
     |████████████████████████████████| 6.5MB 43.5MB/s 
Mounted at /content/drive/


In [ ]:
#importing cleaned file
clean_data=pd.read_csv('/content/drive/MyDrive/UT/Courses/Winter2021/CIV1498/Project/clean_data.csv').drop(columns='Unnamed: 0')
pd.set_option('display.max_columns', None)
clean_data.head()

,Trip Id,Start Time,End Time,Trip Duration,Start Station Id,Start Station Name,End Station Id,End Station Name,Start Latitude,Start Longitude,Start Capacity,End Latitude,End Longitude,End Capacity,User Type,Merge Time,Longitude (x),Latitude (y),Station Name,Climate ID,Year,Month,Day,Time,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Hmdx,Wind Chill,Weather
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,43.660439,-79.385525,11.0,43.653264,-79.382458,35.0,Annual Member,2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359.0,2017.0,1.0,1.0,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,43.645340,-79.409597,26.0,43.639444,-79.425278,15.0,Annual Member,2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359.0,2017.0,1.0,1.0,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,43.646144,-79.377962,47.0,43.662712,-79.379903,19.0,Annual Member,2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359.0,2017.0,1.0,1.0,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),43.638611,-79.414722,19.0,43.651667,-79.384167,31.0,Annual Member,2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359.0,2017.0,1.0,1.0,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St/Queens Quay(Billy Bishop Airport),7010,King St W / Spadina Ave,43.635492,-79.398253,35.0,43.645323,-79.395003,19.0,Annual Member,2017-01-01 00:00:00-05:00,-79.4,43.63,TORONTO CITY CENTRE,6158359.0,2017.0,1.0,1.0,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN


# 2. Data Selection and Grouping

In [ ]:
# select relevant columns for feature selection

def select_columns(data, *columns):
    """Select only columns passed as arguments."""
    return data.loc[:, columns]

used_data = select_columns(clean_data, 'Start Time', 'Temp (°C)', 'Rel Hum (%)', 'Wind Spd (km/h)', 'Weather')

used_data['Trip Count'] = 1
#used_data.head(30)


In [ ]:
#clean weather column to reflect weather conditions

def fix_weather(df):
  
  df_temp = df.copy()
  
  #df_temp['Weather'] = df_temp['Weather'].str.lower()
  df_temp['Weather'].fillna('Clear', inplace = True)
  df_temp.loc[df_temp['Weather'].str.contains("rain", case = False), 'Weather'] = 'Rain'
  df_temp.loc[df_temp['Weather'].str.contains("fog", case = False), 'Weather'] = 'Fog'
  df_temp.loc[df_temp['Weather'].str.contains("haze", case = False), 'Weather'] = 'Haze'
  df_temp.loc[df_temp['Weather'].str.contains("thunder", case = False), 'Weather'] = 'Thunderstorm'
  df_temp.loc[df_temp['Weather'].str.contains("snow", case = False), 'Weather'] = 'Snow'


  return df_temp

weather_fix_data = fix_weather(used_data)

weather_fix_data.head()

,Start Time,Temp (°C),Rel Hum (%),Wind Spd (km/h),Weather,Trip Count
0,2017-01-01 00:03:00-05:00,1.5,69.0,39.0,Clear,1
1,2017-01-01 00:03:00-05:00,1.5,69.0,39.0,Clear,1
2,2017-01-01 00:05:00-05:00,1.5,69.0,39.0,Clear,1
3,2017-01-01 00:09:00-05:00,1.5,69.0,39.0,Clear,1
4,2017-01-01 00:14:00-05:00,1.5,69.0,39.0,Clear,1


In [ ]:
#group all data by hour

def groupbyhour(df):
  
  df_temp = df.copy()

  df_temp['Start Time'] = pd.to_datetime(df_temp['Start Time'], utc=True, errors = 'coerce').dt.tz_convert('EST')

  df_temp = df_temp.groupby([df_temp['Start Time'].dt.floor('h')]).agg(Temperature = ('Temp (°C)', 'mean'),
                                                             Humidity = ('Rel Hum (%)', 'mean'),
                                                             Wind_Speed = ('Wind Spd (km/h)', 'mean'),
                                                             Trip_Count = ('Trip Count', 'sum'),
                                                             Weather = ('Weather','first'))


  return df_temp

grouped_hour = groupbyhour(weather_fix_data)

#drop rows with nan data, as well as all 2016 data
grouped_hour = grouped_hour.dropna(subset = ['Temperature', 'Humidity', 'Wind_Speed']).reset_index()
grouped_hour = grouped_hour.drop(0)
grouped_hour.head()

,Start Time,Temperature,Humidity,Wind_Speed,Trip_Count,Weather
1,2017-01-01 00:00:00-05:00,1.500000,68.000000,37.000000,18,Clear
2,2017-01-01 01:00:00-05:00,1.307692,67.384615,33.846154,13,Clear
3,2017-01-01 02:00:00-05:00,1.066667,67.666667,33.666667,15,Clear
4,2017-01-01 03:00:00-05:00,1.240000,66.600000,33.400000,10,Clear
5,2017-01-01 04:00:00-05:00,1.180000,67.200000,28.800000,5,Clear


In [ ]:
#splitting test data into 70% train, 15% validation, 15% test

from sklearn.model_selection import train_test_split

train, test = train_test_split(grouped_hour, train_size=0.7, 
                               test_size=0.3, random_state=0)
val, test = train_test_split(test, train_size=0.5,
                             test_size=0.5, random_state=0)
print('Train:', train.shape, 'Val:', val.shape, 'Test:', test.shape)

Train: (23287, 6) Val: (4990, 6) Test: (4991, 6)




# 3. Feature Selection

In [ ]:
#import holidays for use
holidays = pd.read_csv('/content/drive/MyDrive/UT/Courses/Winter2021/CIV1498/Project/imported_toronto_holidays')

holidays = holidays.drop(['Unnamed: 0'], axis = 1)

holidays['0'] = pd.to_datetime(holidays['0'], utc=True, errors = 'coerce').dt.tz_convert('EST').dt.date


In [ ]:
#create features for weekends, periods of the day, holidays, and rush hour
def create_time_period(df):
    df_temp = df.copy()
    df_temp['Year'] = df_temp['Start Time'].dt.year
    df_temp['Month'] = df_temp['Start Time'].dt.month
    df_temp['Day'] = df_temp['Start Time'].dt.day
    df_temp['Hour'] = df_temp['Start Time'].dt.hour

    df_temp.loc[:, 'Weekend'] = ((df_temp['Day'] == 5) | (df_temp['Day'] == 6))*1
    bins = np.array([0, 6, 18])
    df_temp.loc[:, 'period'] = np.digitize(df_temp['Hour'], bins)

    df_temp['Holiday'] = df_temp['Start Time'].dt.date.isin(holidays['0'])*1
    df_temp['Rush Hour'] = ((df_temp['Hour'].isin([7,8,9,16,17,18])) & (df_temp['Weekend'] == 0) & (df_temp['Holiday'] == 0))*1
    
    return df_temp


In [ ]:
#one hot encode qualitative weather conditions
def dummy_encode_weather(df):
      categories = ['Clear',
                  'Rain',
                  'Fog',
                  'Haze',
                  'Thunderstorm',
                  'Snow']

      category_type = pd.CategoricalDtype(categories=categories)
      df.loc[:, 'Weather'] = df.loc[:, 'Weather'].astype(category_type)
      df = pd.get_dummies(df, 
                          prefix='Weather:',
                          columns=['Weather'], 
                          drop_first=True)
    
      return df


In [ ]:
#one hot encode periods, since it is not necessarily a linear relationship between periods 1-2-3, more accurate this way
def dummy_encode_period(df):
      categories = [1,2,3]

      category_type = pd.CategoricalDtype(categories=categories)
      df.loc[:, 'period'] = df.loc[:, 'period'].astype(category_type)
      df = pd.get_dummies(df, 
                          prefix='period:',
                          columns=['period'], 
                          drop_first=True)
    
      return df

In [ ]:
#combine all above functions together for easier access
def create_features(df):
    df_temp = df.copy()
    df_temp = create_time_period(df_temp)
    df_temp = dummy_encode_weather(df_temp)
    df_temp = dummy_encode_period(df_temp)
    return df_temp

grouped_hour_train = create_features(train)
grouped_hour_val = create_features(val)

In [ ]:
#Seperate our x and y columns for features and variable to predict (trip_count)
def process_data(df):
    df_temp = df.copy()
    x = df_temp.drop(['Trip_Count','Start Time'], axis = 1)
    y = df_temp.loc[:,'Trip_Count']
    return x, y

x_train , y_train = process_data(grouped_hour_train)
x_val , y_val = process_data(grouped_hour_val)

x_val.head()

,Temperature,Humidity,Wind_Speed,Year,Month,Day,Hour,Weekend,Holiday,Rush Hour,Weather:_Rain,Weather:_Fog,Weather:_Haze,Weather:_Thunderstorm,Weather:_Snow,period:_2,period:_3
12633,13.827273,87.727273,4.909091,2018,6,16,4,0,0,0,0,0,0,0,0,0,0
4389,18.242105,64.973684,4.394737,2017,7,5,0,1,0,0,0,0,0,0,0,0,0
26604,-6.042301,75.846018,15.884956,2020,1,29,8,0,0,1,0,0,0,0,0,1,0
12211,21.196013,73.009967,15.524917,2018,5,29,14,0,0,0,0,0,0,0,0,1,0
4395,19.443038,66.898734,6.493671,2017,7,5,6,1,0,0,0,0,0,0,0,1,0


In [ ]:
#Time to fit the model. Import linear regression and fit curve:

from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept=True)

# 4. Model Fit

In [ ]:
#fit linear model to linear regression model

linear_model.fit(x_train,y_train)
y_fitted = linear_model.predict(x_train)
y_predicted = linear_model.predict(x_val)

In [ ]:
#using the loss function of root mean squared error

from sklearn.metrics import r2_score

def rmse(actual, predicted):
    answer = ((actual-predicted)**2).mean(0)**0.5
    return answer

training_error = rmse(y_train,y_fitted)
val_error = rmse(y_val,y_predicted)

# Print answers
print('Training RMSE: {}'.format(training_error))
print('Validation RMSE: {}'.format(val_error))

Training RMSE: 220.70074582076393
Validation RMSE: 220.09365156020354


In [ ]:
#our RMSE is really high! try polynomial model from scikit with a degree of 3

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
degree=3

polynomial_features = PolynomialFeatures(degree = degree)
x_poly_train = polynomial_features.fit_transform(x_train)
x_poly_val = polynomial_features.fit_transform(x_val)
model = LinearRegression()
model.fit(x_poly_train, y_train)

y_fitted_poly = model.predict(x_poly_train)
y_predicted_poly = model.predict(x_poly_val)

training_error = rmse(y_train,y_fitted_poly)
val_error = rmse(y_val,y_predicted_poly)

print('Training RMSE: {}'.format(training_error))
print('Validation RMSE: {}'.format(val_error))

Training RMSE: 161.99840549870163
Validation RMSE: 167.27560038210797


In [ ]:
#check r2 score for evaluation
from sklearn.metrics import r2_score

coefficient_of_dermination = r2_score(y_predicted, y_val)
coefficient_of_dermination_poly = r2_score(y_predicted_poly, y_val)

print('Simple Validation r2: {} %'.format(coefficient_of_dermination * 100))
print('Polynomial Validation r2: {} %'.format(coefficient_of_dermination_poly * 100))

#As we can see, our polynomial regression greatly outperforms the simple linear regression model

Simple Validation r2: -20.87116441567687 %
Polynomial Validation r2: 54.84854686293053 %



# 5. Testing

In [ ]:
#Time to see how well our model fits to the test data!
model.fit(x_poly_train, y_train)

x_test, y_test = process_data(create_features(test))
x_poly_test = polynomial_features.fit_transform(x_test)

y_predicted = model.predict(x_poly_test)

test_error = rmse(y_predicted, y_test)

coefficient_of_dermination_test = r2_score(y_predicted, y_test)

# Print score
print('Test RMSE: {}'.format(test_error))
print('Test r2: {} %'.format(coefficient_of_dermination_poly * 100))

Test RMSE: 161.31842774049022
Test r2: 54.84854686293053 %


Summary:

1. Data set split into 70-15-15 for training, validation, testing respectively

2. Loss function used is root mean squared error

3. Cross-validation comes from the data set split

4. Features included are as follows: date, hour of the day, weather conditions, holidays, weekends, rush hours, periods of the day, temperature, humidity, and wind speed.

5. Tried two different regression models: simple linear regression and polynomial regression

6. Model is valuated based on 2 metrics: the aforementioned loss function, as well as a r2 score.